# Previous Lesson Overview - Day 8 Pandas_Math

Instructor/Notebook creator: Maria D Hernandez Limon

**AI assistance:** Maria used ChatGPT on *Aug2025* to help with commenting code, brainstorm and design of homework problems.
I reviewed/edited the code, added comments, and validated results with tests/spot checks.

In the previous lesson you learned about dataframes: 

1. Introduction to math and stats with dataframes
2. Introduction to groupby and pivot tables

# Day 09: Seaborn Essentials + SciPy + Maps + Simple Animation 

**Why this day?**  
We level up plots with **Seaborn** and connect your work to **space** (Cartopy/GeoPandas) and **time** (animation).  
And add some **stats** to confirm patterns.


Learning goals (ties to Days 1–8)
- Tidy data → quick plots (`lineplot`, `histplot`/`kdeplot`, `barplot`, faceting).
- Reuse patterns via **plotting functions** (`ax=` in, Axes out).
- Short **Cartopy** map examples (projection + `transform=`).
- Small **GIF** using `FuncAnimation` 



# 0. Setup 

In [ ]:
import pandas as pd
import numpy as np

import re
import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import ticker

#seaborn - more advanced plotting options but always import matplotlib too as some seaborn utility comes from matplot
import seaborn as sns


In [ ]:
#this is the specific directory where the data we want to use is stored
datadirectory = '../data/'

#this is the directory where we want to store the data we finish analyzing
data_out_directory='../output/'

## data prep

In [ ]:
df=pd.read_csv("../data/Temp_data_2010-2020.csv")

# 1) Ensure we have a proper Date 
df['Date'] = pd.to_datetime(df['date'])
# Create Day-of-Year
df['DoY'] = df['Date'].dt.dayofyear
df.head()

# 1. [Seaborn](https://seaborn.pydata.org/)

You can look at the seaborn **[gallery](https://seaborn.pydata.org/examples/index.html)** and look at all the plotting options it offers.

we import as:

`import seaborn as sns` and after we load matplot.

Seaborn has both axes-level plotting functions and figure-level plotting functions.

If a seaborn plot has the following text in the description **'This is an Axes-level function'** then you need to map it on to a figure level object that we can make with matplotlib. If it says **'Figure-level interface for drawing relational plots onto a FacetGrid'** then you don't need to use map on to some figure level object because it already is one.

- **Tidy** (long) data works best (one row = one observation).  
- **Axes-level** funcs (`sns.lineplot`, `sns.barplot`, `sns.histplot`): pass `ax=`, great for subplots.  
- **Figure-level** funcs (`sns.relplot`, `sns.displot`, `sns.catplot`): built-in faceting (`row=`, `col=`), return `FacetGrid`.

## 1.1 Choosing a plot (quick guide)
| Goal | Data shape | First pick |
|---|---|---|
| Trend over time | long, Date/Value/Group | `sns.lineplot` |
| Compare group means + variability | long | `sns.barplot(estimator=np.mean, errorbar="sd")` |
| Distribution | long numeric | `sns.histplot` (+ `kdeplot`) |
| Many panels | long + category facet | `sns.relplot(..., col=..., row=...)` |
| Relationships (a few vars) | long or wide | `sns.pairplot(..., corner=True)` |

## [SEABORN GALLERY](http://seaborn.pydata.org/examples/index.html)

## 1.2 Axes-level vs Figure-level in Seaborn

### A) **Axes-level**: you bring the figure/axes (use ax=)

Rule: Functions like sns.lineplot, sns.barplot, sns.histplot, sns.scatterplot, etc. are Axes-level.
They return a matplotlib.axes.Axes and accept ax=, so you “map” them onto the figure you create with Matplotlib.

In [ ]:
#subset and adjust data
year = 2016
sub = df.loc[df["Year"] == year].copy()
sub = sub.sort_values(["Lake","Date"])

# ---- 1) YOU create the Matplotlib figure & axes grid (because we'll use Axes-level seaborn functions) ----
# 2 rows × 2 cols of subplots; share x/y axes so scales match across panels.
fig, axs = plt.subplots(2, 2, figsize=(10, 6), sharex=True, sharey=True)
# axs is a 2×2 array; ravel() flattens it to a 1D list so we can loop easily.
axs = axs.ravel()
#print(axs)
lakes = ['MI','SU','ER','ON']

# ---- 2) For Axes-level seaborn functions, you must pass ax= to tell them where to draw ----
# these are in axs 
for ax, lake in zip(axs, lakes):
    # Take just this lake’s rows
    d = sub.loc[sub["Lake"] == lake]
    # Add a 7-day rolling mean of Temp_F.
    # rolling(7) looks back up to 7 days; min_periods=3 lets the first few points appear
    # instead of being NaN until 7 days are available.
    d = d.assign(Roll7=d["Temp_F"].rolling(7, min_periods=3).mean())
    #print(d)
    # Draw the line on THIS subplot (Axes) using the Axes-level API.
    # Because this is Axes-level, we *must* pass ax=ax.
    sns.lineplot(data=d, x="Date", y="Roll7", ax=ax)   # <-- Axes-level, mapped onto ax
   # Panel title so we know which lake we are looking at
    ax.set_title(lake)
    # add a horizontal reference line in every facet
    ax.axhline(32, lw=1, ls="--", alpha=0.6, color="red")
    
# A single title for the whole figure (all four panels)
fig.suptitle(f"7-day mean Temp (°F) by Lake — {year}", y=1.02)

#to make fig pretty
plt.tight_layout()
plt.show()

### B) **Figure-level**: Seaborn creates the figure/grid (no ax=)

Rule: Functions like sns.relplot, sns.displot, sns.catplot are Figure-level.
They return a FacetGrid, manage the figure and the subplots for you, and support row=, col=, hue=, height=, aspect=.
You don’t pass ax= to these.

In [ ]:
# Same data slice and rolling temp calculation as above
year = 2016
sub = df.loc[(df["Year"] == year) & (df["Lake"].isin(lakes)),].copy()
sub = sub.sort_values(["Lake","Date"])
sub["Roll7"] = sub.groupby("Lake")["Temp_F"].transform(lambda s: s.rolling(7, min_periods=3).mean())

# ---- 1) Figure-level call: Seaborn creates the Figure + FacetGrid for you ----
# relplot is a *figure-level* function, so we DO NOT pass ax=.
# It builds a grid with one small panel per lake (col="Lake"), wrapped into 2 columns.
g = sns.relplot(
    data=sub, x="Date", y="Roll7",
    col="Lake", col_wrap=2, kind="line",
    height=3, aspect=1.4,
    facet_kws={"sharey": True}
)

# 2) You customize via FacetGrid methods or by looping its axes
g.set_axis_labels("Date", "7-day mean Temp (°F)").set_titles("{col_name}")

# Example: add a horizontal reference line in every facet
for ax in g.axes.flatten():
    ax.axhline(32, lw=1, ls="--", alpha=0.6, color="red")

# Saving: use g.fig (the underlying Matplotlib Figure)
g.fig.suptitle(f"7-day mean Temp (°F) by Lake — {year}", y=1.02)
g.fig.tight_layout()
g.fig.savefig("../output/facet_relplot.png", dpi=150)
plt.show()

| Type             | Examples                                                                  | Returns     | Layout control                                                             | Pass `ax=`? | Best for                                                             |
| ---------------- | ------------------------------------------------------------------------- | ----------- | -------------------------------------------------------------------------- | ----------- | -------------------------------------------------------------------- |
| **Axes-level**   | `lineplot`, `barplot`, `histplot`, `scatterplot`, `boxplot`, `violinplot` | `Axes`      | **You** define with `plt.subplots()`                                       | **Yes**     | Custom figures, mixing plot types, matching other Matplotlib artists |
| **Figure-level** | `relplot`, `displot`, `catplot`                                           | `FacetGrid` | **Seaborn** arranges via `row=`, `col=`, `col_wrap=`, `height=`, `aspect=` | **No**      | Fast faceting, consistent styling and scaling                        |

                                                                                                                                        

## 1.3 Seaborn essentials
### 1.31 Line — monthly climatology (mean ± 1 SD) by Lake
**What:** Seasonal pattern, per lake.  
**Why:** Clean comparison across groups.  
**How:** `groupby → agg → sns.lineplot`, fill a band for ±SD

In [ ]:
#import data
df=pd.read_csv("../data/Temp_data_2010-2020.csv")

#use groupby to find the mean per month and lake 
clim = (df.groupby(['Lake','Month'], as_index=False)
          .agg(mean_T=('Temp_F','mean'), sd_T=('Temp_F','std')))

#initiaize plot 
fig, ax = plt.subplots(figsize=(7,4))

#call searborn line plot 
sns.lineplot(data=clim, x='Month', y='mean_T', hue='Lake', ax=ax, errorbar=None)

#add fill between with a for loop
for lake, sub in clim.groupby('Lake'):
    ax.fill_between(sub['Month'], sub['mean_T']-sub['sd_T'], sub['mean_T']+sub['sd_T'], alpha=0.15)

ax.set(xlabel='Month', ylabel='Temperature (°F)', title='Monthly climatology (mean ± 1 SD)')
ax.set_xticks(range(1,13))
plt.tight_layout()
plt.show()

### Check-in [1]
- Change the markers to triangles and add a grid.
- figure out how to add a grid

In [ ]:

#data selection 
subset = df[df["Lake"].isin(["SU", "ER"])]

#create our plot 
ax = sns.lineplot(data=subset, x="Month", y="Temp", hue="Lake", marker="o")
ax.set(title="Monthly Temp", ylabel="Temp")

#### answer

In [ ]:
subset = df[df["Lake"].isin(["SU", "ER"])]
ax = sns.lineplot(data=subset, x="Month", y="Temp", hue="Lake", marker="^")
ax.set(title="Monthly Temp", ylabel="Temp")
ax.grid(True)

### 1.32 Distributions — histogram + KDE by Season
**What:** Shape of temperatures.  
**Why:** See multimodality & overlaps.  
**How:** `histplot(stat="density")` then optional `kdeplot`.

In [ ]:
df.dtypes

In [ ]:
#import data
df=pd.read_csv("../data/Temp_data_2010-2020.csv")

# 1) Ensure we have a proper Date 
df['Date'] = pd.to_datetime(df['date'])

# 2) Get numeric month
df['Month'] = df['Date'].dt.month

# 3) Map month -> season (meteoro-logical), and make it an ordered categorical for nice plotting
season_map = {
    12: 'Winter', 1: 'Winter', 2: 'Winter',
     3: 'Spring', 4: 'Spring', 5: 'Spring',
     6: 'Summer', 7: 'Summer', 8: 'Summer',
     9: 'Fall',  10: 'Fall',  11: 'Fall'
}
#pd.categorical allows us to specify the order of values in a column -- Like Factor in R
df['Season'] = pd.Categorical(
    df['Month'].map(season_map),
    categories=['Winter','Spring','Summer','Fall'],
    ordered=True
)

# (optional) quick check
print(df['Season'].value_counts())

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))

sns.histplot(data=df, x='Temp_F', hue='Season', bins=24, stat='density', element='step', ax=ax)
sns.kdeplot(data=df, x='Temp_F', hue='Season', common_norm=False, warn_singular=False, ax=ax)

ax.set(title='Distribution of Temp (°F) by Season', xlabel='Temp (°F)')

plt.tight_layout()
plt.show()

### 1.33 Bar — mean Temp by Lake with error bars
**What:** Group means with variability.  
**Why:** Quick summary.  
**How:** `estimator=np.mean`, `errorbar="sd"` or `"se"`.

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))

sns.barplot(data=df, x='Lake', y='Temp_F', 
            estimator=np.mean, errorbar='sd', ax=ax, color="green")

ax.set(title='Mean Temp by Lake (±1 SD)', ylabel='Temp (°F)')
plt.tight_layout()
plt.show()

### 1.34 Pairplot
Rule: sample ≤ 500 rows, `corner=True` to avoid redundancy.
Take a small random sample so the pairplot is fast and readable.
DataFrame.sample(...) returns a random subset of your rows (or columns). 
It’s perfect when a full dataset is too big/noisy to plot.
We cap at 500 rows (or fewer if df has <500). random_state makes the sample reproducible.

In [ ]:
df=pd.read_csv("../data/Temp_data_2010-2020.csv")

# 1) Ensure we have a proper Date 
df['Date'] = pd.to_datetime(df['date'])
# Create Day-of-Year
df['DoY'] = df['Date'].dt.dayofyear
df.head()

In [ ]:
small = df.sample(min(len(df), 500), random_state=0)
# Pairwise relationships among the selected numeric columns.
# - vars=... : which numeric columns to compare pairwise
# - hue='Lake': color points by Lake category
# - corner=True: show only the lower triangle + diagonal (less clutter)
# - plot_kws: tweak the scatter markers (size=18, transparency=0.6) for visibility
sns.pairplot(small, vars=['Temp','Temp_F','DoY'], hue='Lake', corner=True, plot_kws={'s':18, 'alpha':0.6})
plt.show()

### 1.35 Regression with confidence interval

In [ ]:
# One lake, one year: fit a simple curve over the season
sub = df[(df['Year'] == 2018) & (df['Lake'] == 'MI')].dropna(subset=['DoY','Temp_F'])

# 3) Plot a quadratic regression (order=2) to capture the arch-shaped seasonal pattern.
#    - regplot is Axes-level (we pass in an Axes via matplotlib)
#    - ci=95 shows a 95% confidence band around the fitted curve
fig, ax = plt.subplots(figsize=(6,4))
sns.regplot(
    data=sub, x='DoY', y='Temp_F',
    order=2,              # quadratic trend (seasonal arch)
    ci=95,                # confidence band for the fitted curve
    scatter_kws={'s':18, 'alpha':0.4},
    line_kws={'lw':2}
)
ax.set(title='Seasonal trend (MI, 2018)', xlabel='Day of Year', ylabel='Temp (°F)')
plt.tight_layout()
plt.show()

### 1.36 Violin + swarm (distribution per month, one lake)

In [ ]:
lake = 'MI'
sub = df[df['Lake']==lake]

fig, ax = plt.subplots(figsize=(8,4))
sns.violinplot(data=sub, x='Month', y='Temp_F', inner=None, ax=ax, color="blue",alpha=.4)
sns.swarmplot(data=sub.sample(min(len(sub), 300), random_state=1), x='Month', y='Temp_F', size=2, color='k', alpha=.5, ax=ax)
ax.set(title=f'{lake} Temp_F by Month', ylabel='Temp (°F)')

plt.tight_layout()
plt.show()

### 1.37 Check-in [2] 

Import GLERL AMIC data and plot Annual Max Ice Coverga (AMIC)

Goal: Make plots similar to those at https://www.glerl.noaa.gov/data/ice/#historical

The data is in data/great_lakes_maxice.csv, 

make a Seaborn plot: x = Year (time), y = Ice Cover (%), facet per lake, get the bottom ticks to be like those in the GLERL plots. 

Your homework will be to import the raw data for now make a plot. 

In [ ]:
amic = pd.read_csv("../data/great_lakes_maxice.csv")

amic.head()

In [ ]:
# your code for your plot - focus on this if you finish early then work on the tick marks
# you will need this for col_order in sns.relplot
order = ['All_Lakes','Superior','Michigan','Huron','Erie','Ontario']

# look up relplot and figure out how to use it - following the structure of previous examples
# https://seaborn.pydata.org/generated/seaborn.relplot.html
g = sns.relplot(
    #look up the function and the different parameters call what you need
)


#I wrote this for you 
g.set_axis_labels("", "% ice coverage").set_titles("AMIC - Lake {col_name}")
g.set(ylim=(0, 100))


#your code for ticks if you have time 

#headings
g.fig.suptitle("Great Lakes Annual Maximum Ice Cover (AMIC)")
g.fig.tight_layout()
plt.show()

In [ ]:
## the challenging part is getting the x-ticks right - see ref notebook from Day 5, google, chatgpt 



#### answer

In [ ]:
order = ['All_Lakes','Superior','Michigan','Huron','Erie','Ontario']

g = sns.relplot(
    data=amic, x="Year", y="Cover",
    col="Lake", col_order=order, col_wrap=2, kind="line",
    marker="o", dashes=False, linewidth=1,
    height=3, aspect=3.2, color="black",
    facet_kws={"sharex": True, "sharey": True}
)
g.set_axis_labels("", "% ice coverage").set_titles("AMIC - Lake {col_name}")
g.set(ylim=(0, 100))

## code for ticks
min_year = int(amic['Year'].min())
max_year = int(amic['Year'].max())
years = np.arange(min_year, max_year + 1, 1)

# First multiple of 5 >= min_year  → e.g., if min_year=1973, start_major=1975
start_major = ((min_year + 4) // 5) * 5
years_major = np.arange(start_major, max_year + 1, 5)

for ax in g.axes.flatten():
    # Major ticks every 5 years (labeled) starting at 1975 here
    ax.set_xticks(years_major)
    # Minor ticks every year (no labels), so you still "see the ticks per year"
    ax.set_xticks(years, minor=True)

    # Make sure bottom ticks show
    ax.tick_params(axis="x", which="both", bottom=True)

    # Style
    ax.tick_params(axis="x", which="major", length=6, labelrotation=90)
    ax.tick_params(axis="x", which="minor", length=3, labelbottom=False)

g.fig.suptitle("Great Lakes Annual Maximum Ice Cover (AMIC)")
g.fig.tight_layout()
plt.show()

## 1.4 Reusable plotting functions
Why functions? Reuse, composability (`ax=`), and clarity.

### 7.1 `plot_climatology`

In [ ]:
def plot_climatology(df, lakes=None, value='Temp_F', band='sd'):
    """
    Plot monthly climatology (mean ± SD or SEM) for selected lakes.
    
    Parameters
    ----------
    df : DataFrame with columns 'Lake','Month' and the variable (e.g., Temp_F)
    lakes : list of lake names to plot
    value : str, column name of the variable to plot
    band : 'sd' or 'sem(Standard Error of the Mean)' to choose variability measure
    """
    # 1. Filter the data
    use = df[df['Lake'].isin(lakes)]
    
    # 2. Compute mean, SD, SEM per month per lake
    stats = (
        use.groupby(['Lake', 'Month'])[value]
        .agg(['mean', 'std', 'count'])
        .reset_index()
        .rename(columns={'mean': 'mean_val', 'std': 'sd_val', 'count': 'n'})
    )
    stats['sem_val'] = stats['sd_val'] / np.sqrt(stats['n'])
    
    # Choose band
    if band == 'sem':
        stats['band_val'] = stats['sem_val']
    else:
        stats['band_val'] = stats['sd_val']
    
    # 3. Plot
    fig, ax = plt.subplots(figsize=(7, 4))
    for lake in lakes:
        sub = stats[stats['Lake'] == lake]
        ax.plot(sub['Month'], sub['mean_val'], label=lake)
        ax.fill_between(
            sub['Month'],
            sub['mean_val'] - sub['band_val'],
            sub['mean_val'] + sub['band_val'],
            alpha=0.2
        )
    
    ax.set(
        xlabel='Month',
        ylabel=f'{value} (mean ± {band.upper()})',
        title='Monthly Climatology'
    )
    ax.set_xticks(range(1, 13))
    ax.legend()
    plt.tight_layout()

    #note we return ax -- so we could keep modifying it outseide 
    return ax

plot_climatology(df, lakes=['ER','SU'], band='sd')
plt.tight_layout()
plt.show()

In [ ]:
## we can call a new variable to hold our plot and make changes
MI_temp=plot_climatology(df, lakes=['MI'], band='sd')
MI_temp.set_title("Michigan only")

### 7.2 `plot_rolling_trend`

In [ ]:
def plot_rolling_trend(df, lake, year=None, value='Temp_F', window=7, anomaly=False):
    """Plot a rolling-mean time series for a lake (optionally filter to year); 
    optional anomaly vs monthly mean."""
    sub = df.loc[df['Lake']==lake,].copy()

    # if year is teh default then all the data gets selected, otherwise only one year
    if year is not None:
        sub = sub.loc[sub['Year']==year,]
    sub = sub.sort_values('Date')

    ## if anomaly is true then we change the y-values to anomalies 
    ## relative to the monthly mean instead of plotting the raw values -- sub['Value'] changes
    if anomaly == True:
        clim = sub.groupby('Month')[value].transform('mean')
        sub['Value'] = sub[value] - clim
        ylab = f'{value} anomaly (vs monthly mean)'
    else:
        sub['Value'] = sub[value]
        ylab = value
    sub['Roll'] = sub['Value'].rolling(window, min_periods=max(3, window//2)).mean()
    
    
    fig, ax = plt.subplots(figsize=(7,4))
    ax.plot(sub['Date'], sub['Value'], alpha=.35, lw=1, label='daily')
    ax.plot(sub['Date'], sub['Roll'], lw=2, label=f'rolling {window}d')
    ax.set(xlabel='Date', ylabel=ylab, title=f'{lake} — rolling trend' + (f' ({year})' if year else ''))
    ax.legend()
    return ax

## for one specific year, with anomaly as the default so false
plot_rolling_trend(df, 'MI', year=2018, window=7); plt.tight_layout(); plt.show()

## with all the years but anomaly is true
plot_rolling_trend(df, 'HU', window=14, anomaly=True); plt.tight_layout(); plt.show()

### 7.4 `heatmap_year_month`

In [ ]:
def heatmap_year_month(df, lake, stats_wanted ='mean', value='Temp_F'):
    piv = (df.loc[df['Lake']==lake]
             .pivot_table(index='Year', columns='Month', values=value, aggfunc= stats_wanted))
    
    
    plt.figure(figsize=(7,4))
    sns.heatmap(piv, cmap='coolwarm', cbar_kws={'label':f'{stats_wanted} {value}'}, linewidths=.3)
    
    
    plt.title(f'{lake}: {value} by Year × Month')
    plt.xlabel('Month'); plt.ylabel('Year')
    plt.tight_layout()

# Try it: different lake - try different stats - use a list ['max','min']
heatmap_year_month(df, 'ER', stats_wanted = "mean")
plt.show()

# 2.0 [SciPy](https://docs.scipy.org/doc/scipy/reference/index.html)

"SciPy is a collection of mathematical algorithms and convenience functions built on the NumPy extension of Python." 

[Tutorial](https://scipy-lectures.org/packages/statistics/index.html)

I don't have time to cover all of SciPy - below are quick examples.

In [ ]:
##some libraries that allow is to run from stats 
import scipy
from scipy import stats

In [ ]:
temp_data=pd.read_csv(datadirectory+"Temp_data_2010-2020.csv")
temp_data.head()

## 2.1 [t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)

In [ ]:
lake_SU=temp_data.loc[temp_data['Lake']=='SU','Temp_F']
lake_ER=temp_data.loc[temp_data['Lake']=='ER','Temp_F']

#one side- test value of a population mean
print(stats.ttest_1samp(lake_SU, 0))

#two sided- test diff across two populations
print(stats.ttest_ind(lake_SU,lake_ER))

## 2.2 [KS Test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ks_2samp.html)

In [ ]:
from scipy.stats import ks_2samp

lake_SU=temp_data.loc[temp_data['Lake']=='SU','Temp_F']
lake_ER=temp_data.loc[temp_data['Lake']=='ER','Temp_F']

print(stats.ks_2samp(lake_SU, lake_ER))

## 2.3 [One way Anova](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html)

In [ ]:
from scipy.stats import f_oneway

lake_SU=temp_data.loc[temp_data['Lake']=='SU','Temp_F']
lake_MI=temp_data.loc[temp_data['Lake']=='MI','Temp_F']
lake_HU=temp_data.loc[temp_data['Lake']=='HU','Temp_F']
lake_ER=temp_data.loc[temp_data['Lake']=='ER','Temp_F']
lake_ON=temp_data.loc[temp_data['Lake']=='ON','Temp_F']

F, p=f_oneway(lake_SU, lake_MI, lake_HU, lake_ER,lake_ON)
print (F,p)

## 2.4 [Linear Models](https://www.statsmodels.org/stable/index.html)

In [ ]:
from statsmodels.formula.api import ols

selected_lake=temp_data.loc[temp_data['Lake']=='SU']

model = ols("Temp_F ~ Day + Lake", selected_lake).fit()
print(model.summary())

## 2.5 with Plots

In [ ]:
from scipy.stats import linregress

In [ ]:
# 1) Pick a lake and get the data
lake = "Erie"  # change me
sub = amic.loc[amic["Lake"] == lake].sort_values("Year")

x = sub["Year"].to_numpy()
y = sub["Cover"].to_numpy()

# 2) Run SciPy linear regression
res = linregress(x, y)
slope, intercept, r, p, stderr = res.slope, res.intercept, res.rvalue, res.pvalue, res.stderr
r2 = r**2

# 3) Build the fitted line on a dense x-grid (looks nicer)
x_fit = np.linspace(x.min(), x.max(), 200)
y_fit = intercept + slope * x_fit

# 4) Plot data + fit
fig, ax = plt.subplots(figsize=(8,5))
ax.scatter(x, y, s=30, alpha=0.8, label="Annual ice cover")
ax.plot(x_fit, y_fit, lw=2, label="Linear fit", color="red")

# 5) Add an annotation box with stats (axes coords so it stays in corner)
ax.text(
    0.02, 0.68,
    f"Lake {lake}\n"
    f"y = {intercept:.2f} + {slope:.3f}·Year\n"
    f"$R^2$ = {r2:.3f},  p = {p:.3g}",
    transform=ax.transAxes, va="top", ha="left",
    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8)
)

ax.set(
    xlabel="Year",
    ylabel="Ice cover (%)",
    title=f"Trend in annual ice cover — {lake}"
)
ax.grid(True, alpha=0.3)
ax.legend()
plt.show()

## 2.6 Check-in [3]

Question: Are the daily temperature distributions different between Lake Erie (ER) and Lake Superior (SU)?

Use the two-sample Kolmogorov–Smirnov test (scipy.stats.ks_2samp) at α = 0.05.

Show a boxplot (quick summary).

Report: D, p-value, and sample sizes.

Briefly interpret: “Reject/Fail to reject the null that the two distributions are the same.”
(Note: KS tells you they differ, not which is warmer— need medians/means if you want direction.)

In [ ]:
temp=pd.read_csv("../data/Temp_data_2010-2020.csv")
temp.head()

In [ ]:
#select data


#quick boxplots


# KS test (two-sample, nonparametric)


# Annotate stats on the plot -- if time allows


### answer

In [ ]:
#select data
sub = temp.loc[temp["Lake"].isin(["ER", "SU"])].copy()

#quick boxplots
ax = sns.boxplot(data=sub, x="Lake", y="Temp_F")
ax.set(title="Daily Temperature — ER vs SU", xlabel="Lake", ylabel="Temp (°F)")

# KS test (two-sample, nonparametric)
er = sub.loc[sub["Lake"] == "ER", "Temp_F"]
su = sub.loc[sub["Lake"] == "SU", "Temp_F"]

D, p = ks_2samp(er, su, alternative="two-sided", mode="auto")
print(f"KS two-sample: D={D:.3f}, p={p:.3g} | n_ER={er.size}, n_SU={su.size}")

# Annotate stats on the plot
ax.text(
    0.02, 0.95,
    f"D={D:.3f}\np={p:.3g}",
    transform=ax.transAxes, ha="left", va="top",
    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.85)
)

plt.show()

# 3.0 Map basics 

[CartoPy and GeoPandas](https://fneum.github.io/data-science-for-esm/04-workshop-geopandas.html) are two popular ways to make maps with Python. 

CartoPy is great for making publication ready maps with final data where you can chnage every aspect of the map.
GeoPandas is great for data processing, manipulation, then plots. 

The two can work together. 
Below is a short description of each.


## 3.1 CartoPy

**[Cartopy](https://scitools.org.uk/cartopy/docs/latest/)** Python library for maps.

-**[Good Cartopy Intro](https://rabernat.github.io/research_computing_2018/maps-with-cartopy.html)**

-**[Good Tutorial](https://coderzcolumn.com/tutorials/data-science/cartopy-basic-maps-scatter-map-bubble-map-and-connection-map)**

-**[List of Cartopy Projections](https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html)**

[cartopy git page](https://github.com/SciTools/cartopy)

In [ ]:
#for colors
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# this cartopy version was released about 25days ago 8/28 
# so there are many warnings that will eventually be silenced
print (cartopy.__version__)

In [ ]:
#create a new figure and add on via ax
plt.figure(figsize=(8,6))

#we will add a projection of the map to ax then add to it
ax = plt.axes(projection=ccrs.PlateCarree())

plt.show()

In [ ]:
#the size of our figure frame
plt.figure(figsize=(8,6))

#flat projection
ax = plt.axes(projection=ccrs.PlateCarree())

#add coastline
ax.coastlines()
#add countries -- the larger scale for borders triggers a warning 
ax.add_feature(cfeature.BORDERS.with_scale("50m"), linestyle="-")
#add lakes
ax.add_feature(cfeature.LAKES.with_scale("50m"), facecolor="blue")

plt.show()

In [ ]:
plt.figure(figsize=(8,6))

#more realistic spherical projection
ax = plt.axes(projection=ccrs.Orthographic())

#add coastline
ax.coastlines(color="black")
#add ocean
ax.add_feature(cartopy.feature.OCEAN,facecolor='skyblue')

#add countries
ax.add_feature(cfeature.BORDERS,facecolor='white',edgecolor='white')
#add land
ax.add_feature(cartopy.feature.LAND,facecolor="green")


plt.show()

Let's look at the places were Maria has sampled on the Great Lakes

In [ ]:
gl_stations=pd.read_csv(datadirectory+'gl_stations.csv')
gl_stations.head()

**RuntimeWarning** is coming from Natural Earth geometries (most often LAKES or RIVERS) that contain empties/NaNs/invalid rings.

In [ ]:
#select the center points based on the mean of the points I do have
central_lat = gl_stations['LATITUDE'].mean()
central_lon = gl_stations['LONGITUDE'].mean()

#extent is the [west_point,east_point,south_point,north_point]
extent = [gl_stations['LONGITUDE'].min()-2, gl_stations['LONGITUDE'].max()+2,
          gl_stations['LATITUDE'].min()-2, gl_stations['LATITUDE'].max()+2]

#create a figure
plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.Orthographic(central_lon, central_lat))
ax.set_extent(extent)

#add features
ax.add_feature(cartopy.feature.LAKES.with_scale('50m'),edgecolor='black')
ax.add_feature(cartopy.feature.RIVERS.with_scale('50m'))

#setting labels to true will give me labels on x and y axis
ax.gridlines(draw_labels=True)

##add multiple points -- to add multiple points we use scatteplot but we must pass crs.PlateCarree() in transform,
#maybe there are other ways but this is teh only way it works for me
ax.scatter(x=gl_stations.LONGITUDE, y=gl_stations.LATITUDE,color="red",
            s=30,alpha=0.8,transform=ccrs.PlateCarree(),zorder=2)

##### THE Z-ORDER is the order in which layers are shown where 0 is first and the higher is on top, I called zorder to 2 so I could see my points 

#add labels
ax.set_title('LAURENTIAN GREAT LAKES: EPA SAMPLING STATIONS')

#I save most of my figures as .pdf until I'm ready to make manuscript figures
#you can save your plots whiever way you prefer, same commands as other plots 
plt.savefig(data_out_directory+'gl_map.pdf')

Plotting over land

In [ ]:
# Chicago coordinates
lon, lat = -87.6298, 41.8781

# Use PlateCarree (lon/lat degrees)
proj = ccrs.PlateCarree()
fig = plt.figure(figsize=(8,5))
ax = plt.axes(projection=proj)

# Add base map features
ax.add_feature(cfeature.LAND.with_scale('50m'), facecolor="lightgrey")
ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=0.5)
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.5)
ax.add_feature(cfeature.STATES.with_scale("50m"), linewidth=0.4)

# Plot Chicago as a point
ax.plot(lon, lat, marker="o", color="maroon", markersize=8,
        transform=proj, zorder=3)

# Add label
ax.text(lon+3, lat+1, "Chicago", transform=proj,
        fontsize=10, weight="bold")

# Zoom roughly to US
ax.set_extent([-130, -60, 20, 55], crs=proj)

ax.set_title("Where are we?")
plt.show()

## 3.2 GeoPandas 

[GeoPandas Tutorial](https://www.datacamp.com/tutorial/geopandas-tutorial-geospatial-analysis)

[Natural Earth Maps](https://www.naturalearthdata.com/downloads/) #Basefiles you can download

[GeoPandas Projections](https://geopandas.org/en/stable/docs/user_guide/projections.html)

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
gl_stations = pd.read_csv("../data/gl_stations.csv") 
gl_stations.head()

In [ ]:
## we have to reformat the lat/long into points that geopnadas can recognize
gdf = gpd.GeoDataFrame(
    gl_stations,
    geometry=gpd.points_from_xy(gl_stations["LONGITUDE"], gl_stations["LATITUDE"]),
    crs="EPSG:4326"   # lon/lat (WGS84)
)
gdf.head()

In [ ]:
#lakes_url = "https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_lakes.zip"
#lakes = gpd.read_file(lakes_url)

lakes = gpd.read_file('../data/ne_10m_lake.zip')
lakes.head()

In [ ]:
# 1) Choose just the five Laurentian Great Lakes by name
keep = ["Lake Superior","Lake Michigan","Lake Huron","Lake Erie","Lake Ontario"]
great_lakes = lakes[lakes["name"].isin(keep)].copy()

# 2) Define a projection/CRS suitable for the Great Lakes region
#    EPSG:3175 = NAD83 / Great Lakes Albers (equal-area)
crs_gl = "EPSG:3175"  # Great Lakes Albers (equal-area)

# 3) Reproject both polygons (lakes) and points (stations) into this CRS
gl_poly = great_lakes.to_crs(crs_gl) #lake
gl_pts  = gdf.to_crs(crs_gl) #points

# 4) Create a Matplotlib figure + axis
fig, ax = plt.subplots(figsize=(7,6))

# 5) Plot the lakes as filled blue polygons with black borders
gl_poly.plot(ax=ax, color="#cfe8ff", edgecolor="k", linewidth=0.5)

# 6) Plot the station points in red on top
gl_pts.plot(ax=ax, color="crimson", markersize=16, alpha=0.9)

#make pretty and safe 
ax.set_title("LAURENTIAN GREAT LAKES: EPA SAMPLING STATIONS", pad=8)
ax.set_axis_off()
plt.tight_layout()
plt.show()

For land:

In [ ]:
#Basemap
world = gpd.read_file("../data/ne_110m_admin_0_countries.zip")

world.head()

In [ ]:
# Your Chicago point (lon, lat)
chi = gpd.GeoDataFrame(
    {"City": ["Chicago"]},
    geometry=[Point(-87.6298, 41.8781)],   # (lon, lat)
    crs="EPSG:4326" #projection
)

# Plot
fig, ax = plt.subplots(figsize=(8,5))
world.plot(ax=ax, color="lightgrey", edgecolor="white")

# Add the city
chi.plot(ax=ax, color="maroon", markersize=60)

# Add a text label just offset from the point
for x, y, label in zip(chi.geometry.x, chi.geometry.y, chi["City"]):
    ax.text(x+2, y+1, label, fontsize=10, weight="bold")

ax.set_xlim([-130, -60])  # zoom roughly to US
ax.set_ylim([20, 55])
ax.set_title("Where are we?")
plt.show()

## 3.3 Check-in [4]
Make a map with three cities of your choice. 
Fill in the ? in the code below.
Use whatever method you want.

add your points here - the ? should be lists with your values
I want three cities you choose, and use google maps to get lat/lon

```python
my_df=pd.DataFrame({'City':[?],
                   'Lat':[?],
                   'Lon':[?]})
```

In [ ]:
## your map code

### answer

In [ ]:
my_df = pd.DataFrame({
    'City': ['London', 'Seoul', 'Mexico City'],
    'Lat':  [51.5074,   37.5665,  19.4326],
    'Lon':  [-0.1278,  126.9780, -99.1332]
})

proj = ccrs.PlateCarree()  # data are lon/lat degrees
fig = plt.figure(figsize=(8, 4.8))
ax = plt.axes(projection=proj)

# Basemap features
ax.add_feature(cfeature.LAND.with_scale('50m'), facecolor='#f2f2f2')
ax.add_feature(cfeature.OCEAN.with_scale('50m'), facecolor='#e6f3ff')
ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=0.4)
ax.coastlines(resolution='110m', linewidth=0.6)

# Gridlines with labels (cartopy handles °E/°W/°N/°S)
gl = ax.gridlines(draw_labels=True, x_inline=False, 
                  y_inline=False, linewidth=0.5, color='0.7', linestyle=':')
gl.top_labels = False
gl.right_labels = False

# Points (note: transform=PlateCarree because Lon/Lat)
ax.scatter(my_df['Lon'], my_df['Lat'], transform=proj,
           color='crimson', s=40, edgecolor='white', linewidth=0.6, zorder=3)

# Labels
for _, r in my_df.iterrows():
    ax.text(r['Lon'] + 5, r['Lat'] + 3, r['City'], transform=proj,
            fontsize=10, weight='bold')

ax.set_global()  # show the world
ax.set_title('Three Cities (Cartopy)', pad=8)

plt.tight_layout()
plt.show()

In [ ]:
#Basemap
world = gpd.read_file("../data/ne_110m_admin_0_countries.zip")

# 1) Sample cities
my_df = pd.DataFrame({
    'City': ['London', 'Seoul', 'Mexico City'],
    'Lat':  [51.5074,   37.5665,  19.4326],
    'Lon':  [-0.1278,  126.9780, -99.1332]
})

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(
    my_df,
    geometry=gpd.points_from_xy(my_df['Lon'], my_df['Lat']),
    crs="EPSG:4326"   # WGS84 lat/lon
)

# Plot
fig, ax = plt.subplots(figsize=(8,5))
world.plot(ax=ax, color="lightgrey", edgecolor="white")
gdf.plot(ax=ax, color="crimson", markersize=60)

# Add city labels
for x, y, label in zip(gdf.geometry.x, gdf.geometry.y, gdf["City"]):
    ax.text(x+3, y+2, label, fontsize=9, weight="bold")

ax.set_title("Three Cities (GeoPandas)", pad=10)
plt.show()

# 4.0 Quick animation example -> GIF
A gif is a movie, and for movies we need frames that move as a specific speed. We can thus make gifs by combining many frames that we make.

Eric has a great tutorial [here](https://www.ericvc.com/codes/Animation_Example.html)

In [ ]:
from matplotlib.animation import FuncAnimation, PillowWriter

In [ ]:
df=pd.read_csv("../data/Temp_data_2010-2020.csv")

# 1) Ensure we have a proper Date 
df['Date'] = pd.to_datetime(df['date'])
# Create Day-of-Year
df['DoY'] = df['Date'].dt.dayofyear
df.head()

**Select the data**

In [ ]:
# select the dat from frames
lake = 'MI' 
year = 2019
sub = df[(df['Lake']==lake) & (df['Year']==year)].sort_values('Date')

#data arrays - we will make frames with each - this 
x = sub['Date'].to_numpy()
y = sub['Temp_F'].to_numpy()

**Create a blank figure**

In [ ]:

# Create a new figure (canvas) and one subplot (ax)
# figsize sets the width=7 inches and height=4 inches
fig, ax = plt.subplots(figsize=(7,4))

# Set axis labels and a title
# f-string lets you insert variable values (lake, year) into the title
ax.set(xlabel='Date', ylabel='Temp (°F)', title=f'{lake} daily Temp — {year}')

# Set the x-axis and y-axis ranges
# Here x and y are assumed to be arrays of your data
# We expand the y-limits by ±2 for nicer spacing
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min()-2, y.max()+2)

# Initialize a line object (empty plot for now) with line width = 2
# The comma is needed to unpack the single-element tuple that plt.plot returns
(line,) = ax.plot([], [], lw=2)

#show
plt.show()

**create multiple frames and combine**

In [ ]:
# Define the initialization function for the animation
# This runs once at the start: clears the line
def init():
    line.set_data([], [])       # set empty data for x and y
    return (line,)              # return the line object (must be iterable)

# Define the update function for each frame
# i = current frame index (0, 1, 2, …)
def update(i):
    line.set_data(x[:i], y[:i]) # plot data up to frame i
    return (line,)              # return the updated line object
    
# Create the animation object
# - fig: the figure to draw on
# - update: called each frame -- the function we made above 
# - frames=len(x): number of frames in the animation
# - init_func=init: how to reset at start
# - blit=True: only redraw parts that change (faster)
anim = FuncAnimation(fig, update, frames=len(x), init_func=init, blit=True)

# Save the animation as a GIF (12 frames per second)
anim.save('../output/temp_trend.gif', writer=PillowWriter(fps=12))

# Close the figure to avoid duplicate static plot display and save memory
plt.close(fig)

#to confirm our plot is done
#open in browser
print("Saved GIF -> temp_trend.gif")

**Together**

In [ ]:
# Select the data for one lake and year
lake = 'MI' 
year = 2019
sub = df[(df['Lake']==lake) & (df['Year']==year)].sort_values('Date')

# Convert to numpy arrays
x = sub['Date'].to_numpy()
y = sub['Temp_F'].to_numpy()

# Create a new figure (canvas) and one subplot (ax)
fig, ax = plt.subplots(figsize=(7,4))

# Set axis labels and a title
ax.set(xlabel='Date', ylabel='Temp (°F)', title=f'{lake} daily Temp — {year}')

# Set axis ranges
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min()-2, y.max()+2)

# Initialize a line object (empty for now)
(line,) = ax.plot([], [], lw=2)

# Define the initialization function (clears the line at the start)
def init():
    line.set_data([], [])
    return (line,)

# Define the update function (called for each frame)
def update(i):
    line.set_data(x[:i], y[:i])
    return (line,)

# Create the animation object
anim = FuncAnimation(
    fig, update, frames=len(x),
    init_func=init, blit=True
)

# Save the animation as a GIF (12 frames per second)
anim.save('../output/temp_trend.gif', writer=PillowWriter(fps=12))


# Close the figure to avoid duplicate static plot display
plt.close(fig)

# Confirm completion
print("Saved GIF -> temp_trend.gif")


## Check-in [5]

Make a .gif for lake supeior annual max ice cover! The code you need is identical to what you have above you just need to make a couple of changes.

In [ ]:
amic.head()

In [ ]:
# Pick lake & subset

# Figure & empty line

# Animation callbacks

# Create the animation object

# Save the animation as a GIF

# Close the figure to avoid duplicate static plot display

# Confirm completion

### answer

In [ ]:
# Pick lake & subset
lake = "Superior"  
sub = amic.loc[amic["Lake"] == lake].sort_values("Year")

# Data arrays
x = sub["Year"].values
y = sub["Cover"].values  

# Figure & empty line
fig, ax = plt.subplots(figsize=(8,5))
ax.set(xlabel="Year", ylabel="Ice Cover (%)", title=f"Annual Ice Cover — Lake {lake}")

ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min() - 2, y.max() + 2)
(line,) = ax.plot([], [], lw=2, marker="o")

# Animation callbacks
def init():
    line.set_data([], [])
    return (line,)

def update(i):
    line.set_data(x[:i], y[:i])  
    return (line,)

#creat amim
anim = FuncAnimation(fig, update, frames=range(1, len(x)+1), init_func=init, blit=True)

# Save GIF to current folder (no os needed)
anim.save(f'../output/{lake}_amic_trend.gif', writer=PillowWriter(fps=8))
plt.close(fig)
print(f"Saved GIF -> {lake}_amic_trend")


# 5.0 Summary
Today I showed you the following:

1. Explore data with the seaborn plotting library
2. Make figures with AxesSubplot and FacetGrid
3. Using for loops for complicated subplots 
4. Maps
5. Gifs

# 6.0 Self- eval
Use this space to write future notes for your self. 
- What is something you found useful/cool? 
- What is something you don't quite understand and need to follow up with TAs?

# Homework

## Problem 1 - Seaborn Practice

In [ ]:
temp=pd.read_csv(datadirectory+"Temp_data_2010-2020.csv")
temp.head()

### A) for lake Michigan plot boxplots that show the monthly temp distribution 

In [ ]:
## your code using boxplot

#### answer

In [ ]:
lake = "MI"
sub = temp[temp["Lake"] == lake].copy()

ax = sns.boxplot(data=sub, x="Month", y="Temp_F")
ax.set(title=f"Monthly Daily Temperatures — {lake}", xlabel="Month", ylabel="Temp (°F)")
plt.show()

### B) but it would be nice to see all the lakes and only do this once! explore using catplot

In [ ]:
## look up and use sns.catplot

#### answer

In [ ]:
g = sns.catplot(
    data=temp,
    x="Month", y="Temp_F",
    kind="box",
    col="Lake",            # one small panel per lake
    col_wrap=3,            # wrap into rows if many lakes
    height=3.2, aspect=1.2,
    sharey=True
)

## Problem 2 - SciPy

For a chosen lake, test whether January vs July daily temperatures differ (Welch’s t-test). Print t-stat and p-value and a one-line interpretation.

In [ ]:
temp

In [ ]:
from scipy.stats import ttest_ind

## your work 
## use temp data 

### answer

In [ ]:
from scipy.stats import ttest_ind

lake = "ER"
sub = temp[temp["Lake"] == lake].copy()

jan = sub.loc[sub["Month"] == 1, "Temp_F"]
jul = sub.loc[sub["Month"] == 7, "Temp_F"]

tstat, p = ttest_ind(jan, jul, equal_var=False, nan_policy="omit")
print(f"Welch t-test (Jan vs Jul) — {lake}: t={tstat:.3f}, p={p:.3g}")

## Problem 3* - Processing AMIC raw data. [difficult]

The data from great lakes max ice live here: 
url = "https://www.glerl.noaa.gov/data/ice/glicd/dates_AMIC.txt"

import and clean it 
your goal is to recreate great_lakes_maxice.csv that I made for you!!

Work with chatGPT!

In [ ]:
 # Import required libraries (showing students what we need)
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    from scipy import stats

In [ ]:
## your code
## note the number of columns in the first rows is a mess that will be the biggest challenge

### answer

In [ ]:
url = "https://www.glerl.noaa.gov/data/ice/glicd/dates_AMIC.txt"

# Lakes in file order
lakes = ["Superior", "Michigan", "Huron", "Erie", "Ontario", "All_Lakes"]

# Interleaved names: Year, (cover,date) × 6 lakes
names = ["Year"] + [f"{lk}_{k}" for lk in lakes for k in ("cover","date")]
# We only want the first 13 columns: Year + 12 lake tokens (skip trailing duplicate Year)
usecols = list(range(13))

# Read:
# - skiprows=2 → drop the two header lines
# - usecols=range(13) → keep only cols 0..12 (skip trailing Year at col 13)
# - sep=r"\s+" → whitespace-delimited
# - dtype=str first (forgiving), then we'll coerce types
raw = pd.read_csv(
    url,
    sep=r"\s+",
    engine="python",
    header=None,
    names=names,
    skiprows=2,
    usecols=range(13),
    dtype=str,
    skip_blank_lines=True,
)

#drop columns with _date because we don't need them and it will make it easier to melt this table to long format
raw = raw.drop(columns=[c for c in raw.columns if c.endswith("_date")])

raw.head()

In [ ]:
## let use melt
amic = (
    raw.melt(id_vars="Year",            # keep Year as-is
             var_name="Lake",           # old column names go here
             value_name="Cover")        # this will be the new name for col with values
)

# Clean up "Lake" so it’s just the lake name, not "x_cover"
amic["Lake"] = amic["Lake"].str.replace("_cover", "", regex=False)

# Make sure types are right
amic = amic.dropna(subset=["Cover"]).astype({"Year": int, "Cover": float})

amic

## Problem 4* - Let's explore the ice data [difficult]

A major part of data analysis is looking at code others have written and modifying it to do what you need. 

Load the Pokemon data:

In [ ]:
pokemon=pd.read_csv("../data/pokemon.csv")
pokemon.head()

- 1. Analyze the anatomy of this function I wrote to explore the pokemon data:
The code below has many things you have seen before but also some new ones. That's ok. 

You may identify places where I oculd have done something differently or more efficient. 
That's ok! Everyone writes code to their best abilities and as long as it works others can read it then that's good.


- 2. Identify its main components:

Inputs / arguments

Setting up figure + subplots

Looping over y-variables

Scatterplots and regression lines

Adding regression stats as text

Titles, labels

saving output

In [ ]:
def make_pokemon_plot(x_stat='Defense',stats_all=['HP','Attack','Total','Sp. Atk','Sp. Def','Speed']):
    
    """This function will plot some stat vs all the other stat values, with a line, the equation to the line 
    and simple summary stats. The function can be mofidied to plot up to 6 subplots"""

    #set up working space we will add subplots with the for loop
    fig = plt.figure()
    fig.subplots_adjust(hspace=0.3, wspace=0.2)

    #set figure size
    fig.set_size_inches(15,8)

    for count,y_stat in enumerate(stats_all):
        #remember that for position we need three numbers, they need to be intergers
        
        ######these lines will change based on our inputs and edit our layout ####
        if len(stats_all)<=3:
            ax = fig.add_subplot(1, len(stats_all), int(count+1))
        elif len(stats_all)==4:
            ax = fig.add_subplot(2, 2, int(count+1))
        else:
            ax = fig.add_subplot(2, 3, int(count+1))
            
        ###############

        #let's create a boolean command so only the first subplot has a legend for generation
        if count>0:
            #myplot - siple regression plot
            ax=sns.scatterplot(x=x_stat, y=y_stat, data=pokemon,hue='Generation',legend=None)
        else:
            #myplot - siple regression plot
            ax=sns.scatterplot(x=x_stat, y=y_stat, data=pokemon,hue='Generation')

        ax.set_xlabel(x_stat,fontsize=18)
        ax.set_ylabel(y_stat,fontsize=18)

        #####get stats and add them here with a line ####

        #regplot doesn't print any stats so let's use scipy to get some stats and add it to our plots
        ##get stats for a label
        #we use [[]] to select columns from dataframes
        temp=pokemon[[x_stat,y_stat]].dropna()
        #same as pokemon.loc[,[x_stat,y_stat]].dropna()
        slope, intercept, r_value, p_value, std_err = stats.linregress(temp.iloc[:,0],temp.iloc[:,1])
        #print (results)

        ##this code makes the labels in the box
        props=dict(boxstyle='round',facecolor='white',alpha=.4)
        #here is a useful example of .format where you can chane the number format
        textstr='m={:.3f}\nb={:.3f}\n$r^2$={:.2f}\np={:.3}'.format(slope,intercept,r_value**2,p_value) #grabs the values from stats_out
        # here the {} hold the formatting for each value that we will pass to format, the \n means we want a new line
        ax.text(.75,.05,textstr,transform=ax.transAxes,va='bottom',fontsize=11,bbox=props) #change the formatting of the box

        ##### add a line plot

        #this code here allows me to make a line manually
        x1=np.array([temp[x_stat].min(),temp[x_stat].max()])
        y1=slope*x1+intercept
        #this will plot my line on top of the line from regplot
        ax.plot(x1,y1)

        ## add line equation as a title y=mx+b, m=slope, b=intercept
        ax.set_title('y={:.3f}x+{:.2f}'.format(slope,intercept))

    #add a figure title
    plt.suptitle(f'Relationship: Stat vs {x_stat}',fontsize=18)

    #use tight_layout to fix the layout issues we are gaving 
    plt.tight_layout()

    #to save - 
    #the name will be in the fstring and will note the x_stats and how many other stats we called
    plt.savefig(data_out_directory+f"pokemon_scatter{x_stat}vsOthers{len(stats_all)}.pdf")

    #show my figure  
    plt.show()

In [ ]:
## play with the inputs!!
make_pokemon_plot(stats_all=['HP','Total','Attack',],x_stat='Speed')
#make_pokemon_plot(stats_all=['HP','Total','Speed','Defense','Attack'],x_stat='Speed')

Based of my function above write one from the AMIC data. 

Task:

Write a function plot_amic_regressions() with the following behavior:

- Arguments: lakes: list of lake names (e.g., ['Superior','Erie']); years: range years (e.g., (2010,2020)); 

- What it does:

    * Subset the AMIC data to the selected lakes and years.

    * For each selected lake:

    * Plot Year (x) vs Ice Cover % (y).

    * Add a scatterplot and a linear regression line.

    * Display regression stats (slope, intercept, $r^2$, p-value) in a text box.

    * Add the equation of the line as subplot title.

    * Arrange all lakes into subplots.

    * Add a big figure title: "Annual Max Ice Cover Trends"
 
    * Save your plot


Play around with plotting different lakes and years. 
What do you notice about ice trends?
Where you born on a high/low ice year?


In [ ]:
## your code 

def plot_amic_regressions(selected_lakes=['Superior','Michigan','Huron'], 
                         year_range=(1973, 2019),
                         data=amic):
    """
    This function creates regression plots for selected Great Lakes ice coverage over time.
    It shows trends, equations, and statistics similar to the Pokemon example.
    Assumes at least 5 years of data will be included in the analysis.
    
    Parameters:
    - selected_lakes: list of lake names to include in analysis
    - year_range: tuple of (start_year, end_year) to filter data (minimum 5 years)
    - data: DataFrame containing ice coverage data (amic)
    """

    # Filter data by year range 
    
    # Filter by selected lakes - teaching boolean indexing

    # Set up figure working space - following Pokemon example structure

    # Set figure size 


    # Loop through each selected lake 

        # Create subplot layout logic
        # Get data for this specific lake
        # Create scatter plot - basic visualization
        # Set labels with proper font size 


        # Calculate regression statistics - following Pokemon example
        # Remove any missing data points and ensure numeric data types

        # Since we assume at least 5 years, proceed with regression
        # Calculate linear regression 

        # Create text box with statistics - following Pokemon format

        # Add text box to plot with stats 

        #make a line manually
        # add line equation as a title

    # Add overall title

    # Improve layout 

    #save fig

    #show my figure  
    

In [ ]:
# Check output
plot_amic_regressions(
    selected_lakes=['All_Lakes','Superior', 'Erie', 'Michigan','Huron','Ontario'],
    year_range=(1973, 2025))

plot_amic_regressions(
    selected_lakes=['Superior', 'Erie', 'Michigan'],
    year_range=(1980, 2015))

### answer

In [ ]:
def plot_amic_regressions(selected_lakes=['Superior','Michigan','Huron'], 
                         year_range=(1973, 2019),
                         data=amic):
    """
    This function creates regression plots for selected Great Lakes ice coverage over time.
    It shows trends, equations, and statistics similar to the Pokemon example.
    Assumes at least 5 years of data will be included in the analysis.
    
    Parameters:
    - selected_lakes: list of lake names to include in analysis
    - year_range: tuple of (start_year, end_year) to filter data (minimum 5 years)
    - data: DataFrame containing ice coverage data (amic)
    """
    
    # Filter data by year range 
    filtered_data = data.loc[(data['Year'] >= year_range[0]) & 
                        (data['Year'] <= year_range[1]),]
    
    # Filter by selected lakes - teaching boolean indexing
    lake_data = filtered_data.loc[filtered_data['Lake'].isin(selected_lakes),]
    
    # Set up figure working space - following Pokemon example structure
    fig = plt.figure()
    fig.subplots_adjust(hspace=0.4, wspace=0.3)
    # Set figure size 
    fig.set_size_inches(16, 10)
    
    # Loop through each selected lake 
    for count, lake_name in enumerate(selected_lakes):
        
        # Create subplot layout logic 
        if len(selected_lakes) <= 3:
            ax = fig.add_subplot(1, len(selected_lakes), count + 1)
        elif len(selected_lakes) == 4:
            ax = fig.add_subplot(2, 2, count + 1)
        else:
            ax = fig.add_subplot(2, 3, count + 1)
        
        # Get data for this specific lake 
        lake_subset = lake_data[lake_data['Lake'] == lake_name].copy()
        
        # Create scatter plot - basic visualization
        ax.scatter(lake_subset['Year'], lake_subset['Cover'], 
                  color='steelblue', alpha=0.7, s=30)
        
        # Set labels with proper font size 
        ax.set_xlabel('Year', fontsize=14)
        ax.set_ylabel('% Ice Coverage', fontsize=14)
        ax.set_title(f'AMIC - Lake {lake_name}', fontsize=16, fontweight='bold')
        
        # Calculate regression statistics - following Pokemon example
        # Remove any missing data points and ensure numeric data types
        clean_data = lake_subset[['Year', 'Cover']].dropna().copy()
        
        # Since we assume at least 5 years, proceed with regression
        # Calculate linear regression 
        slope, intercept, r_value, p_value, std_err = stats.linregress(clean_data['Year'], clean_data['Cover'])
        
        # Create text box with statistics - following Pokemon format
        textbox_props = dict(boxstyle='round', facecolor='lightblue', alpha=0.8)
        stats_text = f'Slope: {slope:.3f}\nIntercept: {intercept:.1f}\nR²: {r_value**2:.3f}\np-value: {p_value:.3f}'
        
        # Add text box to plot with stats
        ax.text(0.05, 0.45, stats_text, transform=ax.transAxes, 
               verticalalignment='top', fontsize=10, bbox=textbox_props)


        #make a line manually
        x1=np.array([clean_data['Year'].min(), clean_data['Year'].max()])
        y1=slope * x1 + intercept
        #this will plot my line on top of the line from regplot
        ax.plot(x1,y1, color="red")

        #add line equation as a title
        m=slope
        b=intercept
        ax.set_title('y={:.3f}x+{:.2f}'.format(m,b))
        
    
    # Add overall title 
    plt.suptitle("Annual Max Ice Cover Trends",fontsize=18, y=0.98)
    
    # Improve layout and show
    plt.tight_layout()

    #save fig
    plt.savefig("../output/my_ice_trends.png")

    plt.show()


# Check output
#plot_amic_regressions(
#    selected_lakes=['All_Lakes','Superior', 'Erie', 'Michigan','Huron','Ontario'],
#    year_range=(1973, 2025))


In [ ]:
plot_amic_regressions(
    selected_lakes=['All_Lakes','Superior', 'Erie', 'Michigan','Huron','Ontario'],
    year_range=(1973, 2025))

plot_amic_regressions(
    selected_lakes=['Superior', 'Erie', 'Michigan'],
    year_range=(1980, 2015))

## Problem 5 - GeoPandas in the Great Lakes area

### A) Plot lakes in the Great Lakes region above a size threshold

Prompt: From lakes, plot all polygons that:

intersect the Great Lakes bounding box (lon [-170, 5], lat [-50, 83]), and

have area ≥ 1,000 km².
    
Show all lakes in red context, highlight the selected >1,000km2 lakes in a darker color, hide axes, and add a title.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box


# Load lakes
lakes = gpd.read_file("../data/ne_10m_lake.zip")

# Great Lakes bounding box (rough)
gl_bbox = box(-170, 5, -50, 83) 


### your code

# Compute area in km² using an equal-area CRS (Great Lakes Albers)

# Selection: intersects bbox AND area ≥ 1,000 km²

# Plot: context (red) + selected (darker)

### my help :)

# Zoom, tidy, title
ax.set_xlim(-93, -74)
ax.set_ylim(41, 50)
ax.set_axis_off()
ax.set_title("Lakes ≥ 1,000 km² in the Great Lakes Region")
plt.show()

#### answer

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box

# Load lakes
lakes = gpd.read_file("../data/ne_10m_lake.zip")

# Normalize to WGS84 for spatial filtering/plotting
lks = lakes.to_crs(4326)

# Great Lakes bounding box (rough)
gl_bbox = box(-170, 5, -50, 83) 

# Compute area in km² using an equal-area CRS (Great Lakes Albers)
lks_eq = lks.to_crs(3175)  # EPSG:3175
lks = lks.copy()
lks["area_km2"] = lks_eq.geometry.area / 1e6

# Selection: intersects bbox AND area ≥ 1,000 km²
sel = lks[lks.intersects(gl_bbox) & (lks["area_km2"] >= 1000)]

# Plot: context (red) + selected (darker)
fig, ax = plt.subplots(figsize=(7,5))
lks.plot(ax=ax, color="red", edgecolor="white", linewidth=0.5)
sel.plot(ax=ax, color="lightblue", edgecolor="black", linewidth=0.7)

# Zoom, tidy, title
ax.set_xlim(-93, -74)
ax.set_ylim(41, 50)
ax.set_axis_off()
ax.set_title("Lakes ≥ 1,000 km² in the Great Lakes Region")
plt.show()

### B) color your favorite great lake to your favorite color

In [ ]:
## your work 

## use your code from above 

#### answer

In [ ]:
import matplotlib.pyplot as plt

is_sup = lks["name"].str.contains("Superior", case=False, na=False)

fig, ax = plt.subplots(figsize=(8,5))

# all other lakes in black
lks.loc[~is_sup].plot(ax=ax, color="black", edgecolor="black", linewidth=0.4)

# lake superior in pink
lks.loc[is_sup].plot(ax=ax, color="pink", edgecolor="black", linewidth=0.6)

# optional zoom if you have gl_bbox
minx, miny, maxx, maxy = gl_bbox.bounds

# Zoom, tidy, title
ax.set_xlim(-93, -74)
ax.set_ylim(41, 50)
ax.set_axis_off()
ax.set_title("BlackPink in your area! :)")
plt.show()